In [48]:
import csv
import math
import random

In [49]:
# load file and convert strings to float for calculations
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [50]:
# split dataset into train and test datasets
# train - naive bayes' can use to make prediction
# test - use to evaluate the accurecy of the model
# usually 70% - 30% ratio
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [51]:
# seperate trainSet instances by class value to calculate stats of each class
# by creating a map of each class value for list of instances that belong
# to the class and sort the entire dataset of instances into the 
# appropriate list
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [52]:
# calculate the mean of each attribute for a class value
# mean =  central value, central tendancy of data
# Use it as gaussian distribution when calculating probabilities
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [53]:
# calculate stdev by variance and the stdev
def stdev(numbers):
    avg =  mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [54]:
# summarize attributes by class
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [55]:
# summarize attributes by class to make predictions
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [56]:
# make predictions
# 1 calculating gaussian probability density function
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [57]:
# 2 calculating class probability 
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
        return probabilities

In [58]:
# 3 making prediction
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [59]:
# 4 estimating the accuracy
# estimate the accuracy by making predictions in each data instances in our test data
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [60]:
# compare predictions to our class values in test dataset
# and calculate accuracy as a ratio * 100%
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100.0

In [72]:
# call all the funtions to main() in order to predict the accuracy
def main():
    filename = 'datasets/pima-indians-diabetes.csv'
    splitRatio =  0.67
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows in train = {1} and test = {2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    
    # prepare model
    summaries = summarizeByClass(trainingSet)
    
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))
    
main()

Split 768 rows in train = 514 and test = 254 rows
Accuracy: 68.50393700787401%
